In [2]:
from importlib import reload
import tensor
import numpy as np
import copy
import matplotlib.pylab as plt
reload(tensor)

<module 'tensor' from '/home/luca/source/tensor/tensor.py'>

## Ising model
Test dmrg on the ising model hamiltonian

In [ ]:
## Define the MPO for the Ising model in a transverse field

In [3]:
def mpoMatricesIsingModel(J=1,omega=0.1):
    sigmaz=np.array([[1,0],[0,-1]])
    sigmax=np.array([[0,1],[1,0]])
    # central matrix
    O=np.zeros(shape=(8,8))
    O[0:2,0:2]=np.identity(2)
    O[2:2+2,0:2]=sigmaz
    O[4:6,0:2]=sigmax
    O[6:8,0:2]=-omega*sigmax
    O[6:8,2:4]=J*sigmaz
    O[6:8,6:8]=np.identity(2)
    # right vector
    R=np.zeros(shape=(8,2))
    
    R[0:2,]=np.identity(2)
    R[2:4,]=sigmaz
    R[4:6]=sigmax
    R[6:8]=-omega*sigmax
    
    # left vector
    L=np.zeros(shape=(2,8))    
    L[:,0:2]=-omega*sigmax
    L[:,2:4]=J*sigmaz
    L[:,6:8]=np.identity(2)
    
    R=R.reshape((4,2,2))
    L=L.reshape((2,4,2))
    O=O.reshape(4,2,4,2)
    
    
    L2=np.zeros((4,2,4,2))
    L2[0,:,:,:]=L

   
    R2=np.zeros((4,2,4,2))
    R2[:,:,-1,:]=R
    
    return (L2,O,R2)    
def createMpoFromMatrices(L,O,R,nSites=10):
    d=O.shape[1]
    D=O.shape[0]
    op=tensor.mpo(siteDimension=d,bondDimension=D,nSites=nSites)
    op[0].tensor.data=copy.deepcopy(L)
    op[nSites-1].tensor.data=copy.deepcopy(R)
    for i in range(1,nSites-1):
        op[i].tensor.data=copy.deepcopy(O)
        
    return op
def createIsingModelMpo(nSites=10,J=-1,omega=0.1):
    L,O,R=mpoMatricesIsingModel(J=J,omega=omega)
    op=createMpoFromMatrices(L,O,R,nSites=nSites)
    return op
L,O,R=mpoMatricesIsingModel(J=1,omega=0)

In [5]:
reload(tensor)
d=2
D=20
L=20

state=tensor.mps(siteDimension=d,bondDimension=D,nSites=L)
state.orthogonalize()
state.normalize()
op=createIsingModelMpo(nSites=L,J=-1,omega=0)
Heff=tensor.effectiveHamiltonian(mpsState=state,mpoOp=op)
es=Heff.optimize(n=4,outSteps=10)

Energy: (-0.19251090253034464-1.2669543765690532e-16j)
Energy: (-0.6499401774771577+7.885177258451162e-17j)
Energy: (-18.999072352228275+2.614846387148788e-15j)
Energy: (-18.999545351493797-1.8011473622180305e-15j)
Energy: (-19.00000000000008+2.605445402433769e-15j)
Energy: (-19.000000000000085+4.249543261545233e-15j)
Energy: (-19.00000000000016-3.8846688608673434e-16j)
Energy: (-19.000000000000107-5.397781681174882e-15j)
Energy: (-19.000000000000227-1.0462281459348025e-15j)
Energy: (-19.000000000000018-5.433558348655226e-15j)
Energy: (-19.00000000000004-6.9632565346097325e-15j)
Energy: (-19.000000000000124+1.2810445791186593e-15j)
Energy: (-19.000000000000007+6.238597405758925e-15j)
Energy: (-19.000000000000156+1.3387192302300846e-15j)
Energy: (-18.99999842057497+4.816392362697057e-15j)
Energy: (-18.986251994391584-9.297579207668437e-16j)
Energy: (-18.990595487283425+9.71448187694467e-15j)
Energy: (-18.999813054048715+1.1569886088799639e-14j)
Energy: (-18.99999939492417+9.835196842176

In [ ]:
%matplotlib notebook
plt.plot(range(len(es)),es,"or")